In [20]:
import os

In [21]:
template = """#!/bin/bash

########## SBATCH Lines for Resource Request ##########

#SBATCH --nodes=1                   # number of different nodes - could be an exact number or a range of nodes (same as -N)
#SBATCH --mem-per-cpu=8G            # memory required per allocated CPU (or core) - amount of memory (in bytes)
#SBATCH --job-name concept_gen_eva  # you can give your job a name for easier identification (same as -J)
#SBATCH --time=3:59:00              # limit of wall clock time - how long the job will run (same as -t)
#SBATCH --cpus-per-task=2           # number of CPUs (or cores) per task (same as -c)
#SBATCH --ntasks=1                  # number of tasks - how many tasks (nodes) that you require (same as -n)
#SBATCH --gres=gpu:a100:1
#SBATCH -o /mnt/scratch/chenkan4/concept_gen_eva/logfile/resnet/%j.log
#SBATCH -e /mnt/scratch/chenkan4/concept_gen_eva/logfile/resnet/%j.err

########## Command Lines for Job Running ##########

module load Conda/3
conda activate res

ROOT_PATH="/mnt/scratch/chenkan4/concept_gen_eva/src/evaluation/ResNet"
cd ${ROOT_PATH}

JOB_INFO="TOKEN_JOB_INFO"
OBJECT="TOKEN_OBJECT"

COMMAND="python gen_obj.py --object $OBJECT"

#print the information of a job into one file
date >>${ROOT_PATH}/submit_history.log
echo $SLURM_JOB_ID >>${ROOT_PATH}/submit_history.log
echo $JOB_INFO >>${ROOT_PATH}/submit_history.log
echo $COMMAND >>${ROOT_PATH}/submit_history.log

echo "---------------------------------------------------------------" >>${ROOT_PATH}/submit_history.log

echo $JOB_INFO

echo $COMMAND
$COMMAND 1>${ROOT_PATH}/logfile/${SLURM_JOB_ID}_1.log 2>${ROOT_PATH}/logfile/${SLURM_JOB_ID}.err &

wait

echo -e "JobID:$SLURM_JOB_ID \n JOB_INFO: ${JOB_INFO} \n Python_command: \n ${COMMAND}" | mail -s "[Done] ${SLURM_JOB_ID}" harrychen.kr@foxmail.com

date >>${ROOT_PATH}/finish_history.log
echo $SLURM_JOB_ID >>${ROOT_PATH}/finish_history.log
echo $JOB_INFO >>${ROOT_PATH}/finish_history.log
echo $COMMAND >>${ROOT_PATH}/finish_history.log

echo -e "---------------------------------------------------------------" >>${ROOT_PATH}/finish_history.log

scontrol show job $SLURM_JOB_ID     ### write job information to SLURM output file.

"""

In [22]:
objects = 'tench, English springer, cassette player, chain saw, church, French horn, garbage truck, gas pump, golf ball, parachute'.split(', ')

In [23]:
for obj in objects:
    obj_ = obj.replace(' ', '_')
    save_dir = f'sbatch/{obj_}'
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)
    job_info = f"gen_dataset_{obj_}"
    
    for prompt_id in range(5):
        prompt_id = str(prompt_id)
        sbatch = template.replace("TOKEN_JOB_INFO", job_info) \
                    .replace("TOKEN_OBJECT", obj_) \
                    .replace("TOKEN_PROMPT_ID", prompt_id)
        save_path = os.path.join(save_dir, prompt_id)
        with open(save_path, 'w') as file:
            file.write(sbatch)